In [ ]:
from sodapy import Socrata
import seaborn as sns 
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
from sodapy import Socrata
# get api token
client = Socrata("data.melbourne.vic.gov.au", #domain
                'Fake',  #app token
                username="Fake", #api id
                password="Fake")

## First step obtain datasets: 
  - dataset name 
  - id 
  - metadata

In [ ]:
content = []
for a in client.datasets():

  content.append([a['resource']['name'],

  a['resource']['columns_name'],
  #a['resource']['description'],
  a['classification']['categories'],
  a['classification']['domain_category'],
  a['classification']['tags'],
  a['classification']['domain_tags']])



# remove '[]' and join string
content_lst_inital = []
for each_element in content:
  each_element_list = []
  for element in each_element:
    if type(element) == list:
      element = ' '.join(element)
    else:
      pass
    each_element_list.append(element)
  cont= (' ').join(each_element_list)
  content_lst_inital.append(cont)

In [ ]:
content[2]

['Trees, with species and dimensions (Urban Forest)',
 ['Common Name',
  'Located in',
  'CoordinateLocation',
  'Age Description',
  'CoordinateLocation (zip)',
  'CoordinateLocation (city)',
  'CoordinateLocation (state)',
  'Useful Life Expectency',
  'Precinct',
  'CoordinateLocation (address)',
  'Easting',
  'Family',
  'Year Planted',
  'Scientific Name',
  'Latitude',
  'UploadDate',
  'Diameter Breast Height',
  'Useful Life Expectency Value',
  'CoM ID',
  'Longitude',
  'Northing',
  'Genus',
  'Date Planted'],
 ['environment', 'demographics'],
 'Environment',
 [],
 ['accessibility',
  'changemaker',
  'citizenreporter',
  'forest',
  'park',
  'researcher',
  'street',
  'tree',
  'trees',
  'urbanforest',
  '#urbanforest',
  'urban forest']]

In [ ]:
content_lst_inital[2:10]

['Trees, with species and dimensions (Urban Forest) Common Name Located in CoordinateLocation Age Description CoordinateLocation (zip) CoordinateLocation (city) CoordinateLocation (state) Useful Life Expectency Precinct CoordinateLocation (address) Easting Family Year Planted Scientific Name Latitude UploadDate Diameter Breast Height Useful Life Expectency Value CoM ID Longitude Northing Genus Date Planted environment demographics Environment  accessibility changemaker citizenreporter forest park researcher street tree trees urbanforest #urbanforest urban forest',
 'On-street Parking Bay Sensors bay_id st_marker_id status location location (address) location (city) location (state) location (zip) lat lon Municipal boundary transportation economy Transport  parking sensor near real-time vacancy transport travel disability accessibility sensors safemobility',
 'Pay Stay zones linked to street segments between_street_2 pay_stay_zone street between_street_1 street_segment_id transportation

In [ ]:
content_lst = content_lst_inital

# drop ()
content_lst = [re.sub(r'\(.*?\)', '', sent) for sent in content_lst]

#drop _ - 
content_lst = [re.sub('[_-]+', ' ', sent) for sent in content_lst]

content_lst = [re.sub(r'[#&$`\'"]+', '', sent) for sent in content_lst]

#only keep one whitespace 

content_lst = [' '.join(sent.split()) for sent in content_lst]

# drop len(char) = 1 or 2
content_lst = [re.sub(r"((?<=^)|(?<= )).((?=$)|(?= ))", '', sent) for sent in content_lst]

# drop strip
content_lst = [i.strip() for i in content_lst]

content_lst = [re.sub(r'[/]', ' ', sent) for sent in content_lst]

from collections import Counter
def remov_duplicates(input):
 
    # split input string separated by space
    input = input.lower()
    input = input.split(" ")
    input = input
 
    # joins two adjacent elements in iterable way
    for i in range(0, len(input)):
        input[i] = "".join(input[i])
 
    # now create dictionary using counter method
    # which will have strings as key and their
    # frequencies as value
    UniqW = Counter(input)
 
    # joins two adjacent elements in iterable way
    s = " ".join(UniqW.keys())
    return s 



# drop duplicated words and convert string into lowercase
content_lst = [remov_duplicates(i) for i in content_lst]
content_lst = [re.sub(r',','', i) for i in content_lst]
#time
content_lst = [re.sub(r'(\d+:\d+:\d+)','', i )for i in content_lst]

# number  
content_lst = [re.sub(r'\d+\.\d+','', i )for i in content_lst]


def remove_white_sapce(x):
  content = x.split(' ')
  content_list = []
  for i in content:
    if len(i) != 0:
      content_list.append(i)
  result = ' '.join(content) 
  return result 

content_lst = [remove_white_sapce(i) for i in content_lst]
content_lst = [re.sub(r'[\d:]+','', i )for i in content_lst]
content_lst = [re.sub(r"((?<=^)|(?<= )).((?=$)|(?= ))", '', sent) for sent in content_lst]

In [ ]:
words = []
for each_line in content_lst:
  each_line_lst = []
  for each_word in each_line.split(' '):
    each_word = each_word.strip()
    if len(each_word)!= 0:
      each_line_lst.append(each_word)
  words.append(' '.join(each_line_lst))


In [ ]:
words[0:8]

['pedestrian counting system monthly year mdate time day sensor id name hourly counts date month finance transport accessibility cityreactivation covid foot traffic count sensors safemobility flow',
 'tree canopies environment citizenreporter eventsplanner forest urbanforest urban',
 'trees with species and dimensions common name located in coordinatelocation age description useful life expectency precinct easting family year planted scientific latitude uploaddate diameter breast height value com id longitude northing genus date environment demographics accessibility changemaker citizenreporter forest park researcher street tree trees urbanforest urban',
 'on street parking bay sensors id st marker status location lat lon municipal boundary transportation economy transport sensor near real time vacancy travel disability accessibility safemobility',
 'pay stay zones linked to street segments between zone segment id transportation transport parking paystay travel',
 'median house prices 

## Meta Data cleaning Done

In [ ]:
words[0]

'pedestrian counting system monthly date time sensor name hourly counts year month mdate id day finance transport accessibility cityreactivation covid foot traffic count sensors safemobility flow'

## Table

In [ ]:
_name = []
id_ = []
_link = []
for a in client.datasets(): #222
    
    _name.append(a['resource']['name'])
    id_.append(a['resource']['id'])
    _link.append(a['permalink'])
df = pd.DataFrame({'Name':_name,
                'id': id_,
        #        "link": _link
                })
df.loc[:,'content'] = words

In [ ]:
df.head(5)

,Name,id,content
0,Pedestrian Counting System - Monthly (counts p...,b2ak-trbp,pedestrian counting system monthly date time s...
1,Tree canopies 2011 (Urban Forest),y79a-us3f,tree canopies environment citizenreporter even...
2,"Trees, with species and dimensions (Urban Forest)",fp38-wiyy,trees with species and dimensions easting coor...
3,On-street Parking Bay Sensors,vh2v-4nfs,on street parking bay sensors id st marker sta...
4,Pay Stay zones linked to street segments,7q9g-yyvg,pay stay zones linked to street segments zone ...


In [ ]:
!pip install nltk
import nltk
nltk.download('wordnet')
# Install spaCy (run in terminal/prompt)
import sys
!{sys.executable} -m pip install spacy

# Download spaCy's  'en' Model
!{sys.executable} -m spacy download en

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
# Lemmatize with POS Tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()





In [ ]:
each_char = 'trees'

lemmatizer.lemmatize(each_char, get_wordnet_pos(each_char))

'tree'

In [ ]:
nlp_content = []
for each_sentence in words:
  list_1 = []
  for each_char in nltk.word_tokenize(each_sentence):
    tk=lemmatizer.lemmatize(each_char, get_wordnet_pos(each_char))
    list_1.append(tk)
  nlp_content.append(list_1)

In [ ]:
nlp_content[0]

['pedestrian',
 'counting',
 'system',
 'monthly',
 'year',
 'mdate',
 'time',
 'day',
 'sensor',
 'id',
 'name',
 'hourly',
 'count',
 'date',
 'month',
 'finance',
 'transport',
 'accessibility',
 'cityreactivation',
 'covid',
 'foot',
 'traffic',
 'count',
 'sensor',
 'safemobility',
 'flow']

In [ ]:
## check sparsicity 

# make it to sentence 
# nlp_content_string= sentence 
# nlp_content = nlp_content_string.split(' ')
nlp_content_string = [' '.join(i) for i in nlp_content]


#remove built-in english stopwords, convert all words to lowercase, 
#and a word can contain numbers and alphabets of at least length 3 in order to be qualified as a word.
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )


In [ ]:
data_vectorized = vectorizer.fit_transform(nlp_content_string)
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  10.23332399103139 %


In [ ]:
data_vectorized.shape

(223, 128)

In [ ]:
test_model_14 =LatentDirichletAllocation(batch_size=6, n_components = 14,
                        random_state=0,
                        learning_decay=0.7,
                        max_iter= 10,
                        max_doc_update_iter=100,
                        learning_method='batch')
test_model_14.fit(data_vectorized)

LatentDirichletAllocation(batch_size=6, doc_topic_prior=None, evaluate_every=-1,
                          learning_decay=0.7, learning_method='batch',
                          learning_offset=10.0, max_doc_update_iter=100,
                          max_iter=10, mean_change_tol=0.001, n_components=14,
                          n_jobs=None, perp_tol=0.1, random_state=0,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [ ]:
test_model_6 =LatentDirichletAllocation(batch_size=6, n_components = 6,
                        random_state=0,
                        learning_decay=0.7,
                        max_iter= 10,
                        max_doc_update_iter=100,
                        learning_method='batch')
test_model_6.fit(data_vectorized)

LatentDirichletAllocation(batch_size=6, doc_topic_prior=None, evaluate_every=-1,
                          learning_decay=0.7, learning_method='batch',
                          learning_offset=10.0, max_doc_update_iter=100,
                          max_iter=10, mean_change_tol=0.001, n_components=6,
                          n_jobs=None, perp_tol=0.1, random_state=0,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [ ]:


#data is data_vectorized
def return_topic_visual_pro_topic(model, data=data_vectorized):

  # Create Document - Topic Matrix
  lda_output = model.transform(data)
  # column names
  topicnames = ["Topic" + str(i) for i in range(model.n_components)]
  # index names
  docnames = ["dataset" + str(i) for i in range(data.shape[0])]

  # Make the pandas dataframe
  df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

  # Get dominant topic for each document
  dominant_topic = np.argmax(df_document_topic.values, axis=1)
  df_document_topic['dominant_topic'] = dominant_topic

  # Styling
  def color_green(val):
      color = 'green' if val > .1 else 'black'
      return 'color: {col}'.format(col=color)

  def make_bold(val):
      weight = 700 if val > .1 else 400
      return 'font-weight: {weight}'.format(weight=weight)


  # Apply Style
  df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
  

  return  df_document_topics, dominant_topic


In [ ]:
# replace model to see difference 
df_document_topics, dominant_topic  = return_topic_visual_pro_topic(test_model_14, data_vectorized)
df_document_topics


,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,dominant_topic
dataset0,0.010000,0.010000,0.260000,0.340000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.340000,0.010000,0.010000,0.010000,3
dataset1,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.900000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,7
dataset2,0.190000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.420000,0.000000,0.000000,0.240000,0.000000,0.000000,0.120000,7
dataset3,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.210000,0.000000,0.000000,0.000000,0.370000,0.000000,0.000000,0.180000,10
dataset4,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.850000,13
dataset5,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.880000,0.010000,0.010000,0.010000,0.010000,0.010000,8
dataset6,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.900000,13
dataset7,0.000000,0.960000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
dataset8,0.000000,0.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.150000,0.360000,0.080000,0.250000,11
dataset9,0.000000,0.420000,0.050000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.130000,0.000000,0.350000,1


In [ ]:
#topic_n = df_document_topic['dominant_topic']
#df.loc[:,'topic_n'] = topic_n.values
df['content'] = nlp_content_string
df['topic_n'] = dominant_topic

In [ ]:
df[df.loc[:,'topic_n'] == 13]

,Name,id,content,topic_n
4,Pay Stay zones linked to street segments,7q9g-yyvg,pay stay zone link to street segment zone betw...,13
6,On-street Parking Bays,crvt-b4kt,on street parking bay transportation transport...,13
10,Building Permits,r5bg-2hj8,building permit issue date complete by desc of...,13
11,On-street Car Park Bay Restrictions,ntht-5rk7,on street car park bay restriction endtime des...,13
12,Building outlines 2014,qe9w-cym8,building outline housing development property ...,13
14,Development Activity Model Footprints,def8-4wbt,development activity model footprint housing p...,13
17,Street addresses,a7rp-xtya,street address property safe mobility,13
28,Building outlines 2015,pv8y-ihee,building outline housing development property ...,13
38,2018 Building Footprints,pq2z-35fu,building footprint the geom footprint max elev...,13
57,On-street Car Parking Meters with Location,vdsi-4gtj,on street car parking meter with location mete...,13


In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(test_model_14, data_vectorized, vectorizer, mds='tsne')
panel

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:348: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted t

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
6        1.425140 -203.131439       1        1  28.741697
1       93.696953  -55.018154       2        1  14.280351
8      141.694016   25.320429       3        1   7.607361
9     -126.233505    5.739788       4        1   7.152095
11     -59.868134  101.519302       5        1   6.925458
13     151.966629  133.119141       6        1   6.744312
10      12.542690 -101.441658       7        1   5.308047
4       49.372585   57.773476       8        1   5.021417
7      203.505997  -84.541840       9        1   4.200503
0      119.923477 -166.068970      10        1   4.173802
2      -19.315592   -8.882428      11        1   3.655766
5      -95.841057 -109.373474      12        1   2.133940
12      37.554749  165.057098      13        1   2.030417
3      243.648911   32.733547      14        1   2.024834, topic_info=            Term       Freq      Total Category  logprob  loglift
47   environment  79.000000  79.000000  Default  30.0000  30.0000
115    transport  61.000000  61.000000  Default  29.0000  29.0000
127         year  74.000000  74.000000  Default  28.0000  28.0000
87        public  39.000000  39.000000  Default  27.0000  27.0000
95        sensor  40.000000  40.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
43   electricity   0.071853  21.190760  Topic14  -6.7544  -1.7870
60         house   0.071853  12.440082  Topic14  -6.7544  -1.2544
57         green   0.071852  23.543188  Topic14  -6.7544  -1.8923
95        sensor   0.071852  40.383795  Topic14  -6.7544  -2.4319
87        public   0.071852  39.133037  Topic14  -6.7544  -2.4004

[574 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
0         1  0.226571  accessibility
0         2  0.022657  accessibility
0         4  0.158600  accessibility
0         6  0.294542  accessibility
0        11  0.271885  accessibility
...     ...       ...            ...
127       2  0.269533           year
127       3  0.269533           year
127       5  0.080860           year
127      10  0.175196           year
127      14  0.053907           year

[361 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 2, 9, 10, 12, 14, 11, 5, 8, 1, 3, 6, 13, 4])